In [1]:
import numpy as np

In [1]:
def metodo_Euler(f, x0, y0, xf, N):
    h = (xf - x0) / N
    x = np.full(N,x0)
    y = np.full(N,y0)

    for i in range(N):
        y[i+1] = y[i] + h*f(x[i],y[i])
        x[i+1] = x[i] + h
    
    return x, y


def metodo_Heun(f, x0, y0, xf, N):
    h = (xf - x0) / N
    x = np.full(N,x0)
    y = np.full(N,y0)

    for i in range(N):
        k1 = f(x[i],y[i])
        k2 = f(x[i] + h, y[i] + h*k1)
        y[i+1] = y[i] + h*(k1 + k2)/2
        x[i+1] = x[i] + h
    
    return x, y


def metodo_ponto_medio(f, x0, y0, xf, N):
    h = (xf - x0) / N
    x = np.full(N,x0)
    y = np.full(N,y0)

    for i in range(N):
        k1 = f(x[i],y[i])
        k2 = f(x[i] + h/2, y[i] + h/2*k1)
        y[i+1] = y[i] + h*k2
        x[i+1] = x[i] + h

    return x, y


def metodo_RK4(f, x0, y0, xf, N):
    h = (xf - x0) / N
    x = np.full(N,x0)
    y = np.full(N,y0)

    for i in range(N):
        k1 = f(x[i],y[i])
        k2 = f(x[i] + h/2, y[i] + h/2*k1)
        k3 = f(x[i] + h/2, y[i] + h/2*k2)
        k4 = f(x[i] + h, y[i] + h*k3)
        y[i+1] = y[i] + h/6*(k1 + 2*k2 + 2*k3 + k4)
        x[i+1] = x[i] + h

    return x, y


def metodo_adaptativo_RK4(f, x0, y0, xf, hi, tol):
    x = [x0]
    y = [y0]
    h = hi

    while x[-1] < xf:
        k1 = f(x[-1],y[-1])
        k2 = f(x[-1] + h/2, y[-1] + h/2*k1)
        k3 = f(x[-1] + h/2, y[-1] + h/2*k2)
        k4 = f(x[-1] + h, y[-1] + h*k3)
        y1 = y[-1] + h/6*(k1 + 2*k2 + 2*k3 + k4)
        x1 = x[-1] + h

        k1 = f(x1,y1)
        k2 = f(x1 + h/2, y1 + h/2*k1)
        k3 = f(x1 + h/2, y1 + h/2*k2)
        k4 = f(x1 + h, y1 + h*k3)
        y2 = y1 + h/6*(k1 + 2*k2 + 2*k3 + k4)
        x2 = x1 + h

        k1 = f(x[-1],y[-1])
        k2 = f(x[-1] + h, y[-1] + h*k1)
        k3 = f(x[-1] + h, y[-1] + h*k2)
        k4 = f(x[-1] + 2*h, y[-1] + 2*h*k3)
        y3 = y[-1] + h/3*(k1 + 2*k2 + 2*k3 + k4)

        erro = abs(y2 - y3)/30
        rho = tol/erro
        
        if rho > 1:
            x.append(x2)
            y.append(y2)
        else:
            h = min(h*rho**(1/4),2*h)

    return np.array(x), np.array(y)
    

In [4]:
def metodo_shooting(f2, xi, xf, f0i, N, v1, v2, f1=lambda a,b: b, tol=1e-6):
    h = (xf - xi) / N

    def f(r):
        y0 = r[0]
        y1 = r[1]
        fy0 = f1(y0,y1)
        fy1 = f2(y0,y1)

        return np.array([fy0,fy1])
    
    def evolui_r(v):
        r = np.array([f0i,v])

        for t in np.arange(xi,xf,h):
            k1 = h*f(r)
            k2 = h*f(r + k1/2)
            k3 = h*f(r + k2/2)
            k4 = h*f(r + k3)
            r += (k1 + 2*k2 + 2*k3 + k4)/6

        return r[0]
    
    f01 = evolui_r(v1)
    f02 = evolui_r(v2)

    while np.abs(f02 - f01) > tol:
        v = (v1+v2)/2
        f0var = evolui_r(v)

        if f01*f0var > 0:
            v1 = v
            f01 = f0var
        else:
            v2 = v
            f02 = f0var

    return (v1+v2)/2

In [ ]:
def jacobi_laplace(grid, tol):
    """
    Implementação do método de Jacobi para resolver a equação de Laplace.

    Parâmetros:
    - grid (array 2-D): Matriz com as condições de contorno.
    - tol (float): Tolerância para critério de paragem.

    Retorna:
    - array 2-D: Matriz com a solução da equação de Laplace.

    """
    erro = tol + 1
    grid_novo = grid.copy()
    N = grid.shape[0]
    M = grid.shape[1]

    while erro > tol:
        for i in range(1,N-1):
            for j in range(1,M-1):
                grid_novo[i,j] = (grid[i-1,j] + grid[i+1,j] + grid[i,j-1] + grid[i,j+1])/4
                
        erro = np.max(np.abs(grid_novo - grid))
        grid = grid_novo.copy()

    return grid

def jacobi_poisson(grid, f, tol):
    """
    Implementação do método de Jacobi para resolver a equação de Poisson.

    Parâmetros:
    - grid (array 2-D): Matriz com as condições de contorno.
    - f (função): Função que retorna o valor de f(x,y) para um dado ponto.
    - tol (float): Tolerância para critério de paragem.

    Retorna:
    - array 2-D: Matriz com a solução da equação de Poisson.

    """
    erro = tol + 1
    grid_novo = grid.copy()
    N = grid.shape[0]
    M = grid.shape[1]

    while erro > tol:
        for i in range(1,N-1):
            for j in range(1,M-1):
                grid_novo[i,j] = (grid[i-1,j] + grid[i+1,j] + grid[i,j-1] + grid[i,j+1] - f(i,j))/4
                
        erro = np.max(np.abs(grid_novo - grid))
        grid = grid_novo.copy()

    return grid

def gauss_seidel_laplace(grid, tol):
    """
    Implementação do método de Gauss-Seidel para resolver a equação de Laplace.

    Parâmetros:
    - grid (array 2-D): Matriz com as condições de contorno.
    - tol (float): Tolerância para critério de paragem.

    Retorna:
    - array 2-D: Matriz com a solução da equação de Laplace.

    """
    erro = tol + 1
    N = grid.shape[0]
    M = grid.shape[1]

    while erro > tol:
        erro = 0
        for i in range(1,N-1):
            for j in range(1,M-1):
                grid_novo = (grid[i-1,j] + grid[i+1,j] + grid[i,j-1] + grid[i,j+1])/4
                erro = max(erro, abs(grid_novo - grid[i,j]))
                grid[i,j] = grid_novo

    return grid
    
def gauss_seidel_poisson(grid, f, tol):
    """
    Implementação do método de Gauss-Seidel para resolver a equação de Poisson.

    Parâmetros:
    - grid (array 2-D): Matriz com as condições de contorno.
    - f (função): Função que retorna o valor de f(x,y) para um dado ponto.
    - tol (float): Tolerância para critério de paragem.

    Retorna:
    - array 2-D: Matriz com a solução da equação de Poisson.

    """
    erro = tol + 1
    N = grid.shape[0]
    M = grid.shape[1]

    while erro > tol:
        erro = 0
        for i in range(1,N-1):
            for j in range(1,M-1):
                grid_novo = (grid[i-1,j] + grid[i+1,j] + grid[i,j-1] + grid[i,j+1] - f(i,j))/4
                erro = max(erro, abs(grid_novo - grid[i,j]))
                grid[i,j] = grid_novo

    return grid